# Genetic Algorithm and Cellular Automata

## Genetic Algorithm

In [64]:
import ioh
import random
import numpy as np
from algorithm import Algorithm

class GeneticAlgorithm(Algorithm):


    def __init__(self, pop_size=10, dim=10, seed=10, mutation_type = "swap"):
        super().__init__(max_iterations=1000)
        self.mutation_type = mutation_type
        self.dim = dim
        self.seed = seed
        self.dim = dim
        self.population_size = pop_size
        np.random.seed(self.seed)
        self.population = self.generate_population()
        self.y_best = 0.0
        self.x_best : list[int] = []


    def generate_candidate(self):
        """
        Generate an candidate i for the population self.population
        """
        candidate = np.random.randint(2, size=self.dim, dtype=int)
        return candidate

    
    def generate_population(self):
        """
        Generate a candidate for every i in self.population_size
        """
        return [self.generate_candidate() for i in range(self.population_size)]

    
    def print_pop_size(self):
        """
        Print the population size
        """
        print(self.population_size)

       
    def selection(self, k=2):
        """
        * Calculate fitness score/weight for every candidate
        * Randomly select k candidates given selection probabilities
        * return a list of two candidates
        """
        candidates = np.empty(shape=[0, 2])

        for candidate in self.population:
            candidates = np.append(candidates, [[candidate, self.selection_probability(candidate)]], axis=0)

        weights = np.array(candidates[:,1], dtype=float)
        np_candidates = np.array(candidates[:,0])

        return np.random.choice(np_candidates, p = weights, size = 2)

        
    def selection_probability(self, candidate):
        """
        Return candidate fitness score / population fitness score
        """
        return (self.fitness(candidate) / self.population_fitness())

        
    def mutation(self, offspring):
        """
        For every child in the offspring apply either a swap mutatation or an insert mutation
        """
        if self.mutation_type == "swap":
            for child in offspring:
                child = self.swap_mutation(child)
        else:
            for child in offspring:
                child = self.insert_mutation(child)
        return offspring

   
    def swap_mutation(self, child):
        """
        Apply a swap mutation, by randomly selecting 2 genes and swapping them.
        """
        rand_idx = np.random.randint(self.dim,size=2)
        temp = child[rand_idx[1]]
        child[rand_idx[1]] = child[rand_idx[0]]
        child[rand_idx[0]] = temp
        return child

    
    def insert_mutation(self, child):
        """
        * Apply an insert mutation, by randomly selecting an i and a j in the genome.
        * Insert j at position i+1 and delete it's previous index
        """
        child = list(child)
        i_idx = np.random.randint(self.dim-1)
        j_idx = np.random.randint(self.dim)
        if i_idx != j_idx:
            child.insert(i_idx+1, child[j_idx])
            child.pop(j_idx)
        return np.array(child, dtype=int)



    def crossover(self, parents):
        """
        For every child in the offspring apply a swap mutation.
        """
        if(len(parents) != 2):
            raise ValueError("There should be 2 parents")
        offspring = []
        for i in range(self.population_size):
            offspring.append(self.crossover_operator(parents))

        return offspring

    
    def crossover_operator(self, parents):
        """
        * Given two parents, create a child based on a random uniform probability
        * Children are created by a simple treshold. If the probability of a certain index in idx_prob is higher than 0.5, 
        * then a gene from parent A is given, else: parent B
        * This is done for n = self.population_size 
        """
        parentA = np.array(parents[0], dtype=int)
        parentB = np.array(parents[1], dtype=int)
        child = np.empty(shape=[0,1], dtype=int)
        idx_prob = np.random.random_sample(self.dim)
        for i in range(len(idx_prob)):
            if idx_prob[i] > 0.5:
                child = np.append(child, parentA[i])
            else:
                child = np.append(child, parentB[i])
        return child


    
    def fitness(self, candidate):
        """   
        Return the occurrences of 1s
        """
        return np.count_nonzero(candidate)
#         return ioh.problem(candidate)

    
    def population_fitness(self):
        """"
        Calculate the fitness score for the whole population
        Calculation of the fitness score is explained in fitness(self, candidate)
        """
        pop_fitness = 0
        for candidate in self.population:
            pop_fitness += self.fitness(candidate)
        return pop_fitness

    def __call__(self, problem: ioh.problem.Integer) -> None:
        self.dim = problem.meta_data.n_variables
        self.population = self.generate_population()
        while (problem.state.evaluations < self.max_iterations) and (self.x_best != problem.objective.x):
            parents = self.selection(self.population)
            offspring = self.crossover(parents)
            mutated_offspring = self.mutation(offspring)
            for candidate in self.population:
#                 new_y = self.fitness(candidate)
                new_y = problem(candidate)
                if new_y > self.y_best:
                    self.y_best = new_y
                    self.x_best = list(candidate)
                    print('best x: ', self.x_best)
            self.population = mutated_offspring

        problem(self.x_best)
        print('evaluations: ', problem.state.evaluations)
        return problem.state.current_best





In [65]:
def main():

    # Set a random seed in order to get reproducible results
    random.seed(42)

    # Get a problem from the IOHexperimenter environment
    problem: ioh.problem.Integer = ioh.get_problem(1, 1, 5, "Integer")

    # Instantiate the algoritm, you should replace this with your GA implementation
    algorithm = GeneticAlgorithm(mutation_type="insert")

    # Run the algoritm on the problem
    algorithm(problem)

    # Inspect the results
    print("Best solution found:")
    print("".join(map(str, problem.state.current_best.x)))
    print("With an objective value of:", problem.state.current_best.y)
    print()
    
main()

best x:  [1, 0, 0, 1, 0]
best x:  [1, 1, 1, 0, 1]
best x:  [1, 1, 1, 1, 1]
evaluations:  41
Best solution found:
11111
With an objective value of: 5.0



## Cellular Automata and the inversing problem:

In [447]:
import typing
import math
import ioh

from implementation import RandomSearch

class CellularAutomata:
    '''Skeleton CA, you should implement this.'''
    
     
    def __init__(self, init_state: typing.List[int], rule_number: int, neighborhood_radius = 1, length = 10, max_steps = 50, edge=0, base = 2):
        """
        TODO: 
        * Define a rule
        * Given a certain rule, such as 112 -> Convert it to binary
        * Most significant bit is left
        * checks implementeren om te kijken of arrays wel van goede lengte zijn
        * checks implementeren om te kijken of een rule wel valid is
        """
        self.init_state = init_state
        self.time_step = 0
        self.rule_number = rule_number
        self.radius = neighborhood_radius
        self.length = length
        self.max_steps = max_steps
        self.edge = edge
        self.base = base
        self.curr_state = init_state
        self.possible_rules = int(math.pow(2,2*neighborhood_radius+1))
        
        pass
    
    
    def rule_to_base_k(self):
        """
        Convert a rule to a certain base-k representation,
        and then save it to a list
        """
#         if self.base == 2:
        rule_converted = np.base_repr(self.rule_number, base=self.base)
        rule_list = [int(x) for x in str(rule_converted)]
        while(len(rule_list) != self.possible_rules):
            rule_list.insert(0,0)
    
        return rule_list
    
    
    def convert_to_binary(self, num, length):
        """
        Convert a base 10 number to a base 2 representation, given a certain length of te bitstring
        """
        bitstring = np.binary_repr(num, width=length)
        return bitstring
    
    def convert_to_other_base(self, num, length):
        """
        Convert a base 10 number to a base k representation, given a certain length of the output
        """
        converted_num = np.base_repr(num, base=self.base)
        length=self.base * self.radius
    
    def possible_neighborhood_states(self):
        """
        This function returns a list with the possible states, that will to a 1 in the next time step.
        ie, if the list contains 101, it means if for a certain cell C the neighborhood looks like 101 =>
        Cell C will be 1 in timestep t+1
        """
        evaluate_rule = self.rule_to_base_k()
        neighborhood_states = []
        if self.radius == 1:
            neighborhood_state = self.possible_rules
            neighborhood_state_bits = 0
            for i in range(0, self.possible_rules):
                if evaluate_rule[i]:
                    neighborhood_state = self.possible_rules-i-1
                    neighborhood_state_bits = self.convert_to_binary(neighborhood_state, length=self.base *self.radius+1)
                    neighborhood_states.append(neighborhood_state_bits)
        return neighborhood_states
                
        
    def compare_state_rule(self, neighborhood, cell):
        """
        Returns whether neighborhood is contaminated in possible_neighborhood_states()
        Return True if this is the case
        Else return False
        """
        rule_states = self.possible_neighborhood_states()
#         evaluate_rule = self.rule_to_base_k()
        if neighborhood in rule_states:
            return True
        else: return False
        
        
        
    def get_neighbors(self, state):
        
        """
        Given a certain state S
        for every cell in S lookup the neighbors
        For example, given S = [1,0,0,0,0,0,0,0,1,1], base = 2 and radius = 1
        This would return the following dict for cell 8:
        8: {'left': 0, 'cell': 1, 'right': 1},
        """
        left = 0
        right = 0
        neighborhood = {}
        neighborhood_of_cell = {}
        
        if self.radius == 1:
            for cell in range(len(state)):
                if cell == 0:
                    left = 0
                else: left = state[cell-1]
                if cell == len(state)-1:
                    right = 0
                else: right = state[cell+1]
            
                neighborhood_of_cell = {                    
                "left" : left,
                "cell" : state[cell],
                "right" : right 
                }
                neighborhood[cell] = neighborhood_of_cell
        return neighborhood
        
    
    def step(self, state):
        """
        Do a timestep in the cellular automaton
        Given a certain state perform a rule-update (transition)
        Return the new state as a list
        """
        next_state = []
        neighborhood_dict = self.get_neighbors(state)
        for cell in neighborhood_dict:
            left = neighborhood_dict[cell]["left"]
            cell_ = neighborhood_dict[cell]["cell"]
            right = neighborhood_dict[cell]["right"]
            neighbors_bits = f"{str(left)}{str(cell_)}{str(right)}"
            if(self.compare_state_rule(neighbors_bits, cell)):
                next_state.append(1)
            else:
                next_state.append(0)
        
        return next_state


    def __call__(self, c0: typing.List[int], t: int) -> typing.List[int]:
        '''Evaluate for T timesteps. Return Ct for a given C0.'''
        state = self.init_state
        print(state)
        for i in range(30):
            state = self.step(state)
            print(state)
        pass

In [446]:
CA = CellularAutomata(rule_number = 33, base=2, init_state = [1,0,0,0,0,0,0,0,1,1])
CA(c0=[1,0,0,0,0,0,0,0,1,1], t = 300)

# print(CA.possible)
# CA.get_neighbors([1,0,0,0,0,0,0,0,1,1])
CA.rule_to_base_k()
CA.possible_neighborhood_states()
# CA.step([1,0,0,0,0,0,0,0,1,1])


[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]


['101', '000']

In [337]:
int(math.pow(2,3))

8

IndexError: list index out of range